In [50]:
import datetime
import pandas as pd
import tiktoken

def log(string, file="evaluator_output/evaluator_log.txt"):
    print(string)
    with open(file, "a") as log_file:
        log_file.write(string + "\n")

pd.set_option('display.max_colwidth', None)
df = pd.read_csv('data/evaluator_filtered.csv')

datetime_str = datetime.datetime.now().isoformat(sep=" ", timespec="seconds")
log("Experiment " + datetime_str)

instruction = "### instruction ###\nAct as a Russian political joke evaluator. Evaluate the funniness of a political joke using an integer rating from 0 to 3. 0 means the input is not a joke. 1 means the input is a joke but not funny. 3 means the input is very funny.\n"
example_ids = [2, 10, 38, 42]
examples = "### examples ###\n"
for i, id in enumerate(example_ids):
    examples += "Example #" + str(i+1) + "\n<user>: '''" + df.iloc[id,0] + "'''\n<assistant>: '''" + str(df.iloc[id,2]) + "'''"
    if i != len(example_ids) - 1:
        examples += "\n"
log("example data ids:" + str(example_ids))
log("system prompt:\n" + instruction + examples)

def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens
log("system prompt tokens:" + str(num_tokens_from_string(instruction + examples)))

#validate_ids = [3, 11, 39, 43]
validate_ids = [x for x in range(48) if x not in example_ids and x not in range(9, 24)]
log("validation data ids:" + str(validate_ids))
display(df.iloc[validate_ids])

Experiment 2023-11-13 13:22:45
example data ids:[2, 10, 38, 42]
system prompt:
### instruction ###
Act as a Russian political joke evaluator. Evaluate the funniness of a political joke using an integer rating from 0 to 3. 0 means the input is not a joke. 1 means the input is a joke but not funny. 3 means the input is very funny.
### examples ###
Example #1
<user>: '''Q: Is it true that every Soviet soldier dreams of becoming a general? A: No, our soldiers are not that stupid. They know that generals may become MIA even in time of peace.'''
<assistant>: '''3'''
Example #2
<user>: '''Q: Could an atomic bomb destroy our beloved town, Yerevan, with its splendid buildings and beautiful gardens? A: In principle, yes. But Moscow is by far a more beautiful city.'''
<assistant>: '''2'''
Example #3
<user>: '''Q: Is there a difference between capitalism and communism? A: In principle, yes. In capitalism, man exploits man. In communism, it's the reverse.'''
<assistant>: '''1'''
Example #4
<user>: 

,Text,Number of Tokens,Rating out of 3,Rating,Gan's Rating,Yimin's Rating
0,"Q: Is it true that there are two kinds of people serving as deputies of the Supreme Soviet of the USSR, as members of the Supreme Court, and as Soviet diplomats? A: Yes, it is true. One kind is those not capable of anything at all, and the other is those capable of anything whatsoever.",64,3,5,4,5
1,"Q: Why do we need two central newspapers, Pravda (Truth) and Izvestiya (News) if both are organs of the same Party? A: Because in Pravda there is no news, and in Izvestiya there is no truth.",54,3,5,5,4
3,"Q: What is the difference between the Constitutions of the USA and USSR? Both guarantee freedom of speech. A: Yes, and the US Constitution also guarantees freedom after speech.",36,3,5,5,4
4,"Q: Can a son of a General become a Marshal? A: No, because every Marshal also has a son.",24,3,5,5,5
5,"Q: Why did you not broadcast for such a long time? A: We had to make some changes to our staff. The previous broadcaster, while reading an article that contained the words, ""Socialism is nothing as compared with communism,"" made a pause too long after the word ""nothing.""",59,2,4,3,4
6,"Q: Does China have rockets powerful enough to reach the moon? A: They don't need rockets. If the Chinese communist Party ordered, their people just would step on each other's shoulders, and this way they could reach even the sun.",49,2,3,3,2
7,"Q: Why do some people say that Hungarians love the Russians and hate the Americans? A: Because Russians helped Hungarians to get rid of one totalitarian rule, but Americans didn't help to get rid of the other.",45,2,4,4,3
8,"Q: Is it true that American skyscrapers are the tallest in the world? A: Yes, it's true, but on the other hand the Soviet-made transistors are the largest in the world.",43,2,4,3,4
24,"Q: Are the bedbugs also builders of socialism? A: Of course, in their veins flows the workers-and-peasants' blood.",29,2,4,5,2
25,"Q: Why Lenin wore regular shoes, but Stalin wore boots? A: At Lenin's time, Russia was still only ankle-high in shit.",29,2,3,4,2


In [51]:
import os
import openai
from sklearn.metrics import classification_report, mean_squared_error

openai.api_key = os.getenv("OPENAI_API_KEY")

model = "gpt-4-1106-preview"
temperature = 1
top_p = 1
log("model: " + model + ", temperature: " + str(temperature) + ", top_p: " + str(top_p))

output = []
for i in validate_ids:
    response = openai.ChatCompletion.create(
        model="gpt-4-1106-preview",
        messages=[
            {
                "role": "system",
                "content": instruction + examples
            },
            {
                "role": "user",
                "content": df.iloc[i,0]
            }
        ],
        temperature=1,
        max_tokens=512,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    content = response['choices'][0]['message']['content']
    print(content)
    output.append(content)

df2 = df.iloc[validate_ids]
df2.insert(3, 'GPT Rating', output)
df2.to_csv("evaluator_output/" + datetime_str.replace(" ", "_").replace(":", "-") + ".csv", index=False)

log("rmse: " + str(mean_squared_error(df2.iloc[:,2], df2.iloc[:,3], squared=False)))
log(classification_report(df2.iloc[:,2], df2.iloc[:,3].apply(int), labels=range(4), zero_division=0.0))
log("\n\n")


model: gpt-4-1106-preview, temperature: 1, top_p: 1
2
3
2
2
2
1
2
3
2
3
1
1
2
2
1
3
1
1
1
2
2
2
1
2
1
0
0
0
0
0
rmse: 0.7302967433402214
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       0.22      0.40      0.29         5
           2       0.50      0.38      0.43        16
           3       0.25      0.25      0.25         4

    accuracy                           0.47        30
   macro avg       0.49      0.51      0.49        30
weighted avg       0.50      0.47      0.48        30




